# Предобработка текстовых данных

Работа с текстовыми данными обычно начинается с предобработки. Предобработка - это удаление всего лишнего и приведение к нужному формату. Предобработка в большинстве случаев все сводится к использованию стандартных инструментов.

Подготовка текста
- токенизация (слова/n-граммы)
- приведение к нижнему регистру
- лемматизация (приведение слова к начальной/базовой форме - лемме)
- стемминг (нахождение грамматической основы слова - корень слова. Не изменяемая часть при различных вариантах употребления слова)
- удаление стоп-слов
- удаление пунктуации

#### Установите все нужные библиотеки. Подробнее про каждую из них ниже

In [ ]:
# морфологический анализатору Mystem от Яндекса.
# Этот инструмент позволяет проводить морфологический
# анализ русских слов (определение их формы, части речи и т. д.).
!pip install pymystem3 -q

# pymorphy2 предоставляет функционал для морфологического анализа русских слов.
# Опция [fast] указывает на установку быстрой версии библиотеки.
# !pip install pymorphy2[fast]

# razdel предоставляет функции для токенизации
# (разделения текста на отдельные токены, например, слова или предложения)
# русскоязычных текстов.
!pip install razdel -q

# gensim предоставляет инструменты для моделирования темы
# и векторного представления текста.
!pip install gensim -q

# nltk (Natural Language Toolkit) предоставляет множество инструментов
# для работы с естественным языком, таких как токенизация,
# стемминг, разметка речи и многое другое.
!pip install nltk -q

# rusenttokenize предоставляет функции для токенизации русскоязычных текстов
# на предложения.
!pip install rusenttokenize -q

# регулярки
!pip install regex -q

## Токенизация

Предложения нужно разбить на токены. Под токенами обычно понимаются слова, но это могут быть и какие-то более длинные или короткие куски.   
Самый простой способ токенизации -- стандартный питоновский __str.split__ метод.  
По умолчанию он разбивает текст по последовательностям пробелов

In [6]:
'1  2 3'.split(' ')

['1', '', '2', '3']

In [7]:
'1  2 3'.split()

['1', '2', '3']

In [9]:
text = """Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE. Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.

Во-первых, NLI можно использовать для контроля качества генеративных моделей. Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод. Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя". С помощью модели NLI можно проверять, что из X следует Y (то есть в новом тексте нету "отсебятины", придуманной моделью), и что из Y следует X (т.е. вся информация, присутствовавшая в исходном тексте, в новом также отражена).

Во-вторых, с помощью моделей NLI можно находить нетривиальные парафразы и в целом определять смысловую близость текстов. Для русского языка уже существует ряд моделей и датасетов по перефразированию, но кажется, что можно сделать ещё больше и лучше. В статье Improving Paraphrase Detection with the Adversarial Paraphrasing Task предложили считать парафразами такую пару предложений, в которой каждое логически следует из другого – и это весьма логично. Поэтому модели NLI можно использовать и для сбора обучающего корпуса парафраз (и не-парафраз, если стоит задача их детекции), и для фильтрации моделей, генерирующих парафразы.

"""
# текст отсюда - https://habr.com/ru/post/582620/м

In [10]:
text.split()[10:30]

['рассмотреть,',
 'какие',
 'языковые',
 'явления',
 'данная',
 'модель',
 'понимает',
 'хорошо,',
 'а',
 'на',
 'каких',
 '–',
 '"плывёт";',
 'по',
 'этому',
 'принципу',
 'устроены',
 'диагностические',
 'датасеты',
 'SuperGLUE']

Большая часть слов отделяется, но знаки препинания лепятся к словам.
Можно пройтись по всем словам и убрать из них пунктуацию с методом str.strip.

In [11]:
#основные знаки преминания хранятся в питоновском модуле string в punctuation
import string

In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
# в этом списке не хватает кавычек-ёлочек, лапок, длинного тире и многоточия
string.punctuation += '«»—…“”'

In [14]:
[word.strip(string.punctuation) for word in text.split()][10:30]

['рассмотреть',
 'какие',
 'языковые',
 'явления',
 'данная',
 'модель',
 'понимает',
 'хорошо',
 'а',
 'на',
 'каких',
 '–',
 'плывёт',
 'по',
 'этому',
 'принципу',
 'устроены',
 'диагностические',
 'датасеты',
 'SuperGLUE']

Так не будут удаляться дефисы и точки в сокращениях, не разделенных пробелом.
Метод `strip(chars)` в Python удаляет все вхождения символов, указанных в chars, из начала и конца строки.

In [15]:
'как-нибудь'.strip(string.punctuation)

'как-нибудь'

In [ ]:
'т.е.'.strip(string.punctuation)

In [16]:
text_test = "Пример текста с, знаками: препинания!"

# Удаляем знаки препинания из середины строки
text_test_clean = ''.join(el for el in text_test if el not in string.punctuation)

text_test_clean.split(' ')


['Пример', 'текста', 'с', 'знаками', 'препинания']

Например, готовые токенизаторы есть в nltk. Они не удаляют пунктуацию, а выделяют её отдельным токеном.

**wordpunct_tokenizer** разбирает по регулярке - `'\w+|[^\w\s]+'` (попробуйте понять как она работает просто глядя на паттерн)



`\w+:` Эта часть регулярного выражения соответствует одному или более буквенно-цифровым символам (буквы, цифры и знак подчеркивания _). + означает "один или более", а \w соответствует любому буквенно-цифровому символу.

`|:` Этот символ в регулярных выражениях означает "или". Таким образом, регулярное выражение будет искать соответствие либо \w+, либо следующей части.

`[^\w\s]+`: Эта часть регулярного выражения соответствует одному или более символам, которые не являются буквенно-цифровыми (\w) и не являются пробелами (\s). `[^\w\s]` соответствует любому символу, который не является буквенно-цифровым символом или пробелом. + означает "один или более" таких символов.  

То есть регулярка ищет либо последовательность буквенно-цифровых символов (слово), либо последовательность символов, которые не являются буквенно-цифровыми и не являются пробелами

In [17]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [18]:
# wordpunct_tokenize(text)[:10]
word_tokenize(text)[130:150]

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/antontravkin/nltk_data'
    - '/Users/antontravkin/Sites/python_rtk/.venv/nltk_data'
    - '/Users/antontravkin/Sites/python_rtk/.venv/share/nltk_data'
    - '/Users/antontravkin/Sites/python_rtk/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


**word_tokenize** также построен на регулярках, но они там более сложные (учитывается последовательность некоторых
символов, символы начала, конца слова и т.д).

Специально подобранного под русский язык токенизатора там нет,
но и с английским всё работает достаточно хорошо --
сокращения вроде "т.к" собираются в один токен, дефисные слова тоже не разделяются, многоточия тут тоже не отделяются, но это можно поправить.

In [ ]:
word_tokenize(text)[130:150]

In [ ]:
text1 = "Hello, World! It's a beautiful day."
tokens = wordpunct_tokenize(text1)
tokens


In [ ]:
tokens = word_tokenize(text1)
tokens

В генсиме тоже есть функция для токенизации(разбивает текст на токены, преобразуя его в нижний регистр, удаляя знаки препинания и разбивая текст на слова. )

In [ ]:
from gensim.utils import tokenize

In [ ]:
list(tokenize(text, lowercase=True))[30:50]

In [ ]:
list(tokenize(text1, lowercase=True))

И в razdel тоже есть токенизация  


быстрый токенизатор, специально предназначенный для русского языка. Он разбивает текст на токены, учитывая различия в буквенных регистрах и знаках препинания. Токенизация с использованием razdel часто бывает более точной для русского текста.

In [ ]:
from razdel import tokenize as razdel_tokenize

In [ ]:
list(razdel_tokenize(text))[:10]

In [ ]:
[token.text for token in list(razdel_tokenize(text))[:10]]

Работать с регистром тяжело и поэтому можно привести все к нижнему регистру

In [ ]:
[token.text.lower() for token in list(razdel_tokenize(text))[:10]]

задача

задачи

NLP natural language processing

# Нормализация

В последнее время нормализация (т.е. приведение токенов к стандартному виду) используется все реже. Это связано с использованием subword или byte токенизации в топовых моделях. Однако у них есть свои недостатки и забывать про нормализацию пока не стоит.

Два основных вида нормализации - лемматизация и стемминг. Стемминг уже нигде не используется, но его полезно разобрать, чтобы понимать почему нужно использовать лемматизацию

Стемминг - это урезание слова до его "основы" (стема), т.е. такой части, которая является общей для всех словоформ в парадигме слова

На практике стемминг сводится к отбрасыванию частотных окончаний.

Самый известный стеммер - стеммер Портера (или snowball стеммер).
Подробнее про стеммер Портера можно почитать вот тут - <https://medium.com/@eigenein/стеммер-портера-для-русского-языка-d41c38b2d340>  
А совсем подробнее вот тут - <http://snowball.tartarus.org/algorithms/russian/stemmer.html>

Готовые стеммеры для разных языков есть в nltk. Работают они вот так:

In [ ]:
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer('russian')

In [ ]:
[(word, stemmer.stem(word)) for word in word_tokenize(text)][:30]

Недостатки стемминга достаточно очевидные:  
1) с супплетивными формами или редкими окончаниями слова стемминг работать не умеет  
2) к одной основе могут приводится разные слова  
3) к разным основам могут сводиться формы одного слова  
4) приставки не отбрасываются

Супплетивные формы - это словоформы, которые в языке образуются не путем добавления приставок или суффиксов, а путем полной замены корня слова.
 - "Go" и "went"
 - "Is" и "am"
 - "Идти" и "шёл"

# Лемматизация

Лемматизация - это замена словоформы слова в парадигме на какую-то заранее выбранную стадартную форму (лемму).



Например, для разных форм глагола леммой обычно является неопределенная форма (инфинитив), а для существительного форма мужского рода единственного числа. Это позволяет избавиться от недостатков стемминга (будет, был - одна лемма), (пролить, пролом - разные). Однако лемматизация значительно сложнее.

К счастью есть готовые хорошие лемматизаторы. Для русского основых варианта два:Pymorphy и Mystem.


Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Важным достоинством Mystem является то, что он работает не с отдельными словами, а с целым предложением. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию.

### spacy

In [ ]:
!python -m spacy download ru_core_news_sm


In [ ]:
import spacy

# Загружаем модель spaCy
nlp = spacy.load("ru_core_news_sm")  # Для русского языка, можно заменить на en_core_web_sm для английского

text = "Лучшие бегуны бегали по парку с собакой."

doc = nlp(text)

lemmatized_words = [token.lemma_ for token in doc]

print("Лемматизация:", lemmatized_words)
